<a href="https://www.kaggle.com/code/venkateshkonduru/task-2-1-dataanalysis-explor-housemarkindicator?scriptVersionId=100847802" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
parentFolderPath = "../input/housingmarketindicatorcanada"
import os
filenames_list = []
states_list = []
for dirname, _, filenames in os.walk(parentFolderPath):
    for filename in filenames:
        aa = (filename.split("housing-market-indicators-"))[1].split("-")
        #Create states list variable, to use furthur in data preprocessing stage.
        states_list.append(aa[0])
        filepath = os.path.join(dirname, filename)
        filenames_list.append(filepath)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
#Install package to read .xls file
!pip install xlrd

In [21]:
#create category list 
category = ['construction','availablesupply','housingcosts','averagerent','demandinfluences']

# Replace a particular state name in a Python list using a list comprehension
states_list = ['thunder bay' if item == 'thunder' else 'greater sudbury' if item == 'greater' else 'st johns' if item == 'st' else 'prince edward' if item == 'prince' else 'nova scotia' if item == 'nova' else 'saint john' if item == 'saint' else 'british columbia' if item == 'british' else 'qubec cma' if item == 'qubec' else 'new brunswick' if item == 'new' else item for item in states_list]

**Based on the initial Data Analysis:** 
* Each dataset has four categories (Construction, Housing Costs, Average Rent, Demand Influences). 
* Each category has its own datapoints. Each datapoint has missing value(NA) on the year range scale.
* The Data Preprocessing divided into two sections:
    
    **Section 1:** Do dataset level EDA. Read the statewise xls data. Split each category into four datasets. Each dataset is added with two new columns (category,state). category indicates which category thr datapoint belongs and state indicates to which state "xls" belongs too.export as csv file
    
    **Section 2:** Do point level EDA( to treat missing values either with Mean or Median imputation technique). Join datasets to single dataset

The mentioned sections ben followed for all states dataset.

In [22]:
def get_dataframe_split_index(df_clean):
    AS_header_index = 0
    HC_header_index = 0
    DI_header_index = 0
    AR_header_index = 0
    for index in range(df_clean.first_valid_index(),len(df_clean)):
        if(df_clean["Name"][index] == "Available Supply"):
            AS_header_index = index;
        if(df_clean["Name"][index] == "Housing Costs"):
            HC_header_index = index;
        if(df_clean["Name"][index].__contains__("Average rent")):
            AR_header_index = index;
        if(df_clean["Name"][index] == "Demand Influences"):
            DI_header_index = index;
    return AS_header_index,HC_header_index,DI_header_index,AR_header_index

In [23]:
df_dict ={}
df_last2rows = {}

In [24]:
def read_excel_():
    for i in range(len(filenames_list)):
        df = pd.read_excel(filenames_list[i],header = 3,na_filter = False)
        df[['category','state']] = ''
        df.rename(columns = {'Unnamed: 0':'Name'}, inplace = True)
        df_clean = df.drop([0,1], axis=0)
        AvailableSupply_header_index, HousingCosts_header_index,DemansInfluences_header_index,AverageRent_header_index = get_dataframe_split_index(df_clean)
        df_Construction = df.iloc[2:AvailableSupply_header_index,:]
        #df_Construction.shape
        df_Construction['category']= category[0]
        df_Construction['state']=states_list[i]
        df_AvailableSupply = df.iloc[(AvailableSupply_header_index+1):HousingCosts_header_index,:]
        # df_AvailableSupply.shape
        df_AvailableSupply['category']= category[1]
        df_AvailableSupply['state']=states_list[i]
        df_HousingCosts = df.iloc[(HousingCosts_header_index+1):AverageRent_header_index,:]
        # df_HousingCosts.shape
        df_HousingCosts['category']= category[2]
        df_HousingCosts['state']=states_list[i]
        df_AverageRent = df.iloc[(AverageRent_header_index+1):DemansInfluences_header_index,:]
        # df_AverageRent.shape
        df_AverageRent['category']= category[3]
        df_AverageRent['state']=states_list[i]
        df_DemandInfluences = df.iloc[(DemansInfluences_header_index+1):(df_clean.last_valid_index()-3),:]
        # df_DemandInfluences.shape
        df_DemandInfluences['category']= category[4]
        df_DemandInfluences['state']=states_list[i]
        df_section1 = pd.concat([df_Construction, df_AvailableSupply, df_HousingCosts, df_AverageRent, df_DemandInfluences], ignore_index=True)
        df_dict[states_list[i]] = df_section1.T

In [25]:
read_excel_()

In [26]:
for state in states_list:
    new_header = df_dict[state].iloc[0] #grab the first row for the header
    df_dict[state] = df_dict[state][1:] #take the data less the header row
    df_dict[state].columns = new_header #set the header row as the df header
    columnss = df_dict[state].columns
    last_element = columnss[-1]
    if last_element.__contains__("1 Homeowner"):
        df_dict[state].drop([last_element], inplace=True, axis=1) #Drop the unneeded legend column
    df_dict[state].replace('NA', np.nan,inplace=True) #replace NA with NaN
    df_dict[state].replace('**', np.nan,inplace=True) #replace ** with NaN
    df_last2rows[state] = df_dict[state][-2:]
    df_dict[state].drop(df_dict[state].tail(2).index,inplace=True)
    df_dict[state] = df_dict[state].astype(float) #Convert datatype of column
    df_dict[state].interpolate(method='linear',limit_direction = 'both',inplace=True) #nterpolation
    df_dict[state].append(df_last2rows[state])
    df_dict[state].to_csv('./'+state+'_section1_.csv')  

**Next Step:** Based on the discussion, I will continue this steps
* Convert Categorical Values into Numerical Values Either using one-hot encoding or function to replace Categorical value to numerical value.
* Based on the inference '0' value will get imputed either with mean of the column or median of the column.

**Data Collection Reference**
* https://www.cmhc-schl.gc.ca/en/professionals/housing-markets-data-and-research/housing-data/data-tables/housing-market-indicators